In [3]:
import time
import os
# os.chdir("..")
from copy import deepcopy
import torch
import cv2
import numpy as np
import matplotlib.cm as cm
from src.utils.plotting import make_matching_figure
# from src.utils.plotting import make_matching_figure_2
print(torch.cuda.is_available())
# os.environ["CUDA_VISIBLE_DEVICES"] = "1, 0"
from src.loftr import LoFTR, default_cfg
_default_cfg = deepcopy(default_cfg)
_default_cfg['coarse']['temp_bug_fix'] = True # set to False when using the old ckpt
_default_cfg['coarse']['npe'] = [832, 832, 832, 832]
print(_default_cfg)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
matcher = LoFTR(config=_default_cfg)
matcher.load_state_dict(torch.load('./weights/eloftr_outdoor.ckpt')['state_dict'])
matcher.eval().to(device)

/home/baizy/anaconda3/envs/GSAD/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
{'backbone_type': 'RepVGG', 'align_corner': False, 'resolution': (8, 1), 'fine_window_size': 8, 'fp16': False, 'replace_nan': False, 'eval_times': 1, 'backbone': {'block_dims': [64, 128, 256]}, 'coarse': {'d_model': 256, 'd_ffn': 256, 'nhead': 8, 'layer_names': ['self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross'], 'agg_size0': 4, 'agg_size1': 4, 'no_flash': False, 'rope': True, 'npe': [832, 832, 832, 832], 'temp_bug_fix': True}, 'match_coarse': {'thr': 0.2, 'border_rm': 2, 'dsmax_temperature': 0.1, 'train_coarse_percent': 0.2, 'train_pad_num_gt_min': 200, 'sparse_spvs': True, 'skip_softmax': False, 'fp16matmul': False}, 'match_fine': {'sparse_spvs': True, 'local_regress_temperature': 1.0, 'local_regress_slicedim': 8}}


LoFTR(
  (backbone): RepVGG_8_1_align(
    (layer0): RepVGGBlock(
      (nonlinearity): ReLU()
      (se): Identity()
      (rbr_dense): Sequential(
        (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (rbr_1x1): Sequential(
        (conv): Conv2d(1, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (layer1): ModuleList(
      (0): RepVGGBlock(
        (nonlinearity): ReLU()
        (se): Identity()
        (rbr_identity): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rbr_dense): Sequential(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [4]:
# Load example images
img0_pth = "/data/baizeyu/GSPose/data/MIP/Box/train/0000.png"
img1_pth = "/data/baizeyu/GSPose/data/MIP/Box/train/0003.png"
#img0_pth = "D:\jx\data\040-1-014231-514-004490.jpg"
#img1_pth = "D:\jx\data\040-2-014231-514-004490.jpg"
img0_raw = cv2.imread(img0_pth, cv2.IMREAD_GRAYSCALE) # 灰度模式读取
img1_raw = cv2.imread(img1_pth, cv2.IMREAD_GRAYSCALE)
# img0_raw = cv2.resize(img0_raw, (400, 400))
# img1_raw = cv2.resize(img1_raw, (400, 400))

img0 = torch.from_numpy(img0_raw)[None][None].to(device) / 255.
img1 = torch.from_numpy(img1_raw)[None][None].to(device)/ 255.
batch = {'image0': img0, 'image1': img1}

In [10]:
# Inference with LoFTR and get prediction
with torch.no_grad():
    start = time.time()
    matcher(batch)
    end = time.time()
    print("Time: ", end - start)
    mkpts0 = batch['mkpts0_f'].cpu().numpy()
    mkpts1 = batch['mkpts1_f'].cpu().numpy()
    mconf = batch['mconf'].cpu().numpy()

Time:  0.07581782341003418


In [6]:
mkpts0.shape

(725, 2)